In [ ]:
import numpy as np
from scipy.integrate import quad
import scipy.optimize as scpo
import numpy as np
import pandas as pd
from numpy import exp, sqrt
from geneticalgorithm import geneticalgorithm as ga
import random

In [ ]:
call_df = pd.read_csv('')
History = pd.read_csv('')

In [ ]:
def Heston_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v) ** 2 -
                  sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
        / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = (r * u * 1j * T + (c1 / sigma_v ** 2) *
          ((kappa_v - rho * sigma_v * u * 1j + c2) * T -
          2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2 *
          ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

def Bates_call(row):
    u=1000
    S0 = row.Closing_price
    K = row.stike
    T = row.nDiff / 365
    r = row.r / 100
    kappa_v=row.kappa_v
    theta_v=row.theta_v
    sigma_v=row.sigma_v
    rho=row.rho
    v0=row.v0
    lamb=row.lamb
    delta=row.delta
    mu=row.mu
    int_value = quad(lambda u:
                     Bates_int_func(u, S0, K, T, r, kappa_v, theta_v,
                                  sigma_v, rho, v0, lamb, mu, delta),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value

def Bates_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0,
                 lamb, mu, delta):
    char_func_value = Bates_char_func(u - 1j * 0.5, T, r, kappa_v, theta_v,
                                    sigma_v, rho, v0, lamb, mu, delta)
    int_func_value = 1 / (u ** 2 + 0.25) \
        * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    return int_func_value


def Merton_char_func(u, T, lamb, mu, delta):
    omega = -lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
    char_func_value = np.exp((1j * u * omega + lamb *
        (np.exp(1j * u * mu - u ** 2 * delta ** 2 * 0.5) - 1)) * T)
    return char_func_value


def Bates_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0,
                  lamb, mu, delta):
    BCC1 = Heston_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0)
    BCC2 = Merton_char_func(u, T, lamb, mu, delta)
    return BCC1 * BCC2

# Non-linear LS calibration

In [ ]:
def Bates_call_value(S0,K,T,r,kappa_v, theta_v, sigma_v, rho, v0,
                 lamb, mu, delta):
    u=1000
    
    int_value = quad(lambda u:
                     Bates_int_func(u, S0, K, T, r, kappa_v, theta_v,
                                  sigma_v, rho, v0, lamb, mu, delta),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value

def Bates(x,kappa_v, theta_v, sigma_v, rho, v0,
                 lamb, mu, delta):
    K,S0,r,T=x
    u=1000
    r=r/100
    T=T/365
    return f2(S0,K,T,r,kappa_v, theta_v, sigma_v, rho, v0,
                 lamb, mu, delta)

f2 = np.vectorize(Bates_call_value) #Vectorize pricer

In [ ]:
init_vals = [0.06, 0.04, 0.2, 0.6, 0.04,1,0.2,0.2]
bounds = ( [0.001, 0.001, 0.001, -0.9, 0.001,0.001,0.001,0.001] , [1,1,1,1,1,5,1,1])
params_Bates = scpo.curve_fit(Bates, (call_df.stike.values,call_df.Closing_price.values,call_df.r.values,call_df.nDiff.values),call_df.Close.values, p0=init_vals, bounds=bounds)

In [ ]:
call_df[['kappa_v', 'theta_v', 'sigma_v', 'rho', 'v0',
                  'lamb','mu', 'delta']] = params_Bates[0] 

Bates_params = call_df[['kappa_v', 'theta_v', 'sigma_v', 'rho', 'v0',
                  'lamb','mu', 'delta']].iloc[0]

Bates_params.to_csv('../Data/Bates_params.csv') #save params

In [ ]:
call_df['Bates'] = call_df.apply(Bates_call, axis=1)

# Returns for model with Non-lienar LS calibrated params

In [ ]:
kappa, theta, sigma, v0, rho, lambdaJ, muJ, sigmaJ= Bates_params
S0 = History[0].close
NT = len(History)
NS = 1
r = 0.001
q = 0
dt = 3/365 #Mat/NT
# Expected value of k, and drift term
kappa2 = exp(muJ) - 1
drift = r - q - lambdaJ*kappa2
# Initialize the variance and stock processes
V = np.zeros((NT,NS))
S = np.zeros((NT,NS))
# Starting values for the variance and stock processes
S[0,:] = S0       # Spot price
V[0,:] = v0       # Heston v0 initial variance

# Generate the paths
for i in range(NS):
    for t in range(1,NT):
    # Generate two dependent N(0,1) variables with correlation rho
        Zv = random.normalvariate(0,1)
        Zs = rho*Zv + sqrt(1-rho**2)*random.normalvariate(0,1)
        # Milstein discretization for the variance.
        V[t,i] = V[t-1,i] + kappa*(theta-V[t-1,i])*dt \
        + sigma*sqrt(V[t-1,i]*dt)*Zv \
        + (1/4)*sigma**2*dt*(Zv**2-1)
    # Simulate the lognormal jumps
        J = 0
        if lambdaJ != 0:
            Nt = np.random.poisson(lambdaJ*dt)
            if Nt > 0:
                for x in range(Nt):
                    J = J + random.normalvariate(muJ - sigmaJ**2/2,sigmaJ)

    # Discretize the log stock price
        S[t,i] = S[t-1,i]*exp((drift-V[t-1,i]/2)*dt + J + sqrt(V[t-1,i]*dt)*Zs)
S.to_csv('./Bates-stock')

# Genetic Algorithm Calibration

In [ ]:
varbound=np.array([[0.01,0.5],[0.01,0.2],[0.15,0.9],[0.01,0.4],[-1,1],[0.01,5],[0,0.1],[0.01,0.1]])
algorithm_param = {'max_num_iteration': 200,\
            'population_size':2700,\
            'mutation_probability':0.1,\
            'elit_ratio': 0.01,\
            'crossover_probability': 0.5,\
            'parents_portion': 0.3,\
            'crossover_type':'uniform',\
            'max_iteration_without_improv':23}

In [ ]:
def Bates_GA(y):
    global S
    def f(x):
        kappa, theta, sigma, v0, rho, lambdaJ, muJ, sigmaJ=x
        S0 = y[0]
        NT = len(y)
        NS = 1
        r = 0.001
        q = 0
        dt = 3/365 #Mat/NT
        # Expected value of k, and drift term
        kappa2 = exp(muJ) - 1
        drift = r - q - lambdaJ*kappa2
        # Initialize the variance and stock processes
        V = np.zeros((NT,NS))
        S = np.zeros((NT,NS))
        # Starting values for the variance and stock processes
        S[0,:] = S0       # Spot price
        V[0,:] = v0       # Heston v0 initial variance

        # Generate the paths
        for i in range(NS):
            for t in range(1,NT):
            # Generate two dependent N(0,1) variables with correlation rho
                Zv = random.normalvariate(0,1)
                Zs = rho*Zv + sqrt(1-rho**2)*random.normalvariate(0,1)
                # Milstein discretization for the variance.
                V[t,i] = V[t-1,i] + kappa*(theta-V[t-1,i])*dt \
                + sigma*sqrt(V[t-1,i]*dt)*Zv \
                + (1/4)*sigma**2*dt*(Zv**2-1)
            # Simulate the lognormal jumps
                J = 0
                if lambdaJ != 0:
                    Nt = np.random.poisson(lambdaJ*dt)
                    if Nt > 0:
                        for x in range(Nt):
                            J = J + random.normalvariate(muJ - sigmaJ**2/2,sigmaJ)

            # Discretize the log stock price
                S[t,i] = S[t-1,i]*exp((drift-V[t-1,i]/2)*dt + J + sqrt(V[t-1,i]*dt)*Zs)
        S.to_csv('./Bates-stock-GA')
        return np.mean((S.reshape(-1)-y)**2)

    model=ga(function=f,\
            dimension=8,\
            variable_type='real',\
            variable_boundaries=varbound,\
            algorithm_parameters=algorithm_param,
         convergence_curve=False,
         progress_bar=True)
    model.run()
    return model.best_variable


In [ ]:
call_df[['kappa_v', 'theta_v','sigma_v', 'v0', 'rho', 'lamb', 'mu', 'delta']] = Bates_GA(History.Close.values)

In [ ]:
call_df['Bates-GA'] = call_df.apply(Bates_call, axis=1)

# Save dataframe

In [ ]:
call_df.to_csv('./Bates-input-output')

# Metrics

In [ ]:
from .utilties import utilties

In [ ]:
line1 = utilties.error_metrics(call_df['Close'], call_df['Bates'])

In [ ]:
line2 = utilties.error_metrics(call_df['Close'], call_df['Bates-GA'])

In [ ]:
for line in ([*line1], [*line2]):
  print('& {:.2f} & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% \\\\'.format(*line))